In [ ]:
import pandas as pd
import numpy as np

base_path = "Proyectos_Datos_Edicion_01_EVM"
df = pd.read_csv(f"{base_path}/data/linea_base_subestacion.csv")

# 1. Simulación de avance con variabilidad lógica hasta el Mes 8
def simular_avance(row):
    m = row["Mes"]
    if m > 8: return 0, 0 # Futuro

    # Factores base con ruido
    spi_base = 0.81 + np.random.uniform(-0.05, 0.10)
    cpi_base = 0.82 + np.random.uniform(-0.08, 0.05)

    ev = row["PV"] * spi_base
    ac = ev / cpi_base
    return round(ev, 0), round(ac, 0)

df[["EV", "AC"]] = df.apply(lambda r: simular_avance(r), axis=1, result_type="expand")

# 2. Ajuste Matemático para cumplir KPI exacto en Mes 8
m8_mask = df["Mes"] == 8
ev_m8 = df.loc[m8_mask, "EV"].sum()
pv_m8 = df.loc[m8_mask, "PV"].sum()

adj_spi = (pv_m8 * 0.81) / ev_m8
df.loc[m8_mask, "EV"] = (df.loc[m8_mask, "EV"] * adj_spi).round(0)

ev_m8_adj = df.loc[m8_mask, "EV"].sum()
ac_m8 = df.loc[m8_mask, "AC"].sum()
adj_cpi = (ev_m8_adj / 0.82) / ac_m8
df.loc[m8_mask, "AC"] = (df.loc[m8_mask, "AC"] * adj_cpi).round(0)

df.to_csv(f"{base_path}/data/historico_real.csv", index=False)
print(">>> Script 2: Histórico real con CPI=0.82 y SPI=0.81 generado.")